# Method 1
## COCO dataset - annotated foreground

In [1]:
import os
import numpy as np
import tensorflow as tf
import skimage.io as io
import skimage.transform as transform
import matplotlib
import matplotlib.pyplot as plt
from pycocotools.coco import COCO

%matplotlib inline
matplotlib.use('Agg')

### Function 1: Foreground Augmentation - Increase data variety

In [2]:
# Random rotation, resizing, translation and flipping

def foregroundAug(foreground):
    
    angle = np.random.randint(-10,10)*(np.pi/180.0) # Convert to radians
    zoom = np.random.random()*0.4 + 0.8             # Scale in range [0.8,1.2)
    
    t_x = np.random.randint(0, int(foreground.shape[1]/3))
    t_y = np.random.randint(0, int(foreground.shape[0]/3))

    tform = transform.AffineTransform(scale=(zoom,zoom),
                                rotation=angle,
                                translation=(t_x, t_y))
    foreground = transform.warp(foreground, tform.inverse)

    # Random horizontal flip with 0.5 probability
    if(np.random.randint(0,100)>=50):
        foreground = foreground[:, ::-1]
        
    return foreground

### Function 2: Get the mask of the augmented foreground instance

In [3]:
def getForegroundMask(foreground):
    mask_new = foreground.copy()[:,:,0]
    mask_new[mask_new>0] = 1
    return mask_new

### Function 3: Paste the foreground onto the background

In [4]:
def compose(foreground, mask, background):
    # Resize background to match the size of foreground
    background = transform.resize(background, foreground.shape[:2])

    # Subtract the foreground area from the background
    background = background*(1 - mask.reshape(foreground.shape[0], foreground.shape[1], 1))

    # Paste the foreground on the background
    composed_image = background + foreground 
  
    return composed_image

## Step 1: Load Data via COCO API

In [5]:
annFile = 'data1/instances_val2017.json'

coco=COCO(annFile)

loading annotations into memory...
Done (t=0.71s)
creating index...
index created!


## Step 2: Data Category Selection

In [6]:
catIds = coco.getCatIds(catNms=['cat'])
imgIds = coco.getImgIds(catIds=catIds)

print("Number of images containing all filter classes:", len(imgIds))

Number of images containing all filter classes: 184


## Step 3 - 7

In [7]:
# Iteratively read foreground images from COCO
for i in range(len(imgIds)):      
#---------------------- Step 3: Extract Foreground ----------------------#
    img = coco.loadImgs(imgIds[i])[0]
    print(i, img['file_name'])
    I = io.imread(img['coco_url'])/255.0
    
    # Load annotations via COCO API
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    
    # Select proper annotated instances with size and segmentation
    if (len(anns[0]['segmentation']) > 1 or 
        len(anns) > 1 or
        anns[0]['area'] < 15000 or 
        anns[0]['area'] > 80000):
        continue
    
    #coco.showAnns(anns)
    mask = np.zeros((img['height'],img['width']))
    for n in range(len(anns)):
        mask = np.maximum(coco.annToMask(anns[n]), mask)

    mask = mask.reshape(I.shape[0], I.shape[1], 1)

    foreground = mask*I
    
    #-------------------- Step 4: Read Background --------------------#
    
    DIR = "data1/background/cloister/"
    imagePaths = [os.path.join(DIR, f) for f in os.listdir(DIR)]
    if DIR + '.DS_Store' in imagePaths:
        imagePaths.remove(DIR + '.DS_Store')
    
    for j in range(len(imagePaths)):   # Iteratively read background images
        
        background = io.imread(imagePaths[j],plugin='matplotlib')/255.0
        background = np.array(background)
        
        if background.shape[2] != foreground.shape[2]:   # Should be with same dimension
            continue
        #-------------------- Step 5: Augment Foreground --------------------#            
        foreground_new = foregroundAug(foreground)
        
        #-------------------- Step 6: Get Foreground Mask--------------------#  
        mask_new = getForegroundMask(foreground_new)
        plt.imsave("data1/output/mask/{}_{}.png".format(i, j), mask_new)
        
        #-------------------- Step 7: Paste --------------------#  
        composed_image = compose(foreground_new, mask_new, background)
        #axes[i].imshow(composed_image)
    
    
        composed_image = tf.clip_by_value(composed_image, 0.0, 1.0)
        #print(type(composed_image))
        
        plt.imsave("data1/output/image/{}_{}.png".format(i, j), 
                   composed_image.numpy())      
      
    
        #plt.imshow(np.uint8(composed_image))
        #plt.axis('off')
        #fig.savefig("./output/dining_room/{}_{}.png".format(img['file_name'][6:12],imagePaths[j][25:-4]), 
        #            bbox_inches = 'tight', pad_inches = 0)
           
    
    

0 000000416256.jpg
1 000000269314.jpg
2 000000223747.jpg
3 000000245764.jpg
4 000000458255.jpg
5 000000047121.jpg
6 000000411665.jpg
7 000000312340.jpg
8 000000119828.jpg
9 000000098839.jpg
10 000000222235.jpg
11 000000290843.jpg
12 000000533536.jpg
13 000000545826.jpg
14 000000343076.jpg
15 000000402473.jpg
16 000000320554.jpg
17 000000460841.jpg
18 000000494634.jpg
19 000000101420.jpg
20 000000357941.jpg
21 000000080949.jpg
22 000000071226.jpg
23 000000131131.jpg
24 000000366141.jpg
25 000000063552.jpg
26 000000401991.jpg
27 000000261706.jpg
28 000000469067.jpg
29 000000416330.jpg
30 000000107087.jpg
31 000000277584.jpg
32 000000077396.jpg
33 000000078420.jpg
34 000000475732.jpg
35 000000519764.jpg
36 000000327769.jpg
37 000000116825.jpg
38 000000078426.jpg
39 000000424545.jpg
40 000000211042.jpg
41 000000181859.jpg
42 000000456292.jpg
43 000000435299.jpg
44 000000329319.jpg
45 000000279145.jpg
46 000000157807.jpg
47 000000316015.jpg
48 000000169076.jpg
49 000000366199.jpg
50 0000000

#### Code Reference: 
https://github.com/virafpatrawala/Synthetic-Image-Datasets